In [1]:
from hello import hello3

h = hello3.open_hello('71.189.82.196:88')
h.login()

data = {}
for group in ("agitation", "temperature", "ph", "do"):
    td = h.getTrendData("15min", group)
    data[group] = td['last']

len(data.keys())

%matplotlib
import matplotlib.pyplot as plt

from datetime import datetime
fig = plt.figure()
ax = fig.add_subplot(111)
for g in data:
    d = data[g]
    x,y = list(zip(*d))
    t = [datetime.fromtimestamp(z/1000) for z in x]
    ax.plot(x,y, label=g)

ax.legend()

In [55]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def process_data(ws, cell, all_chart):
    cell.Offset(1,2).EntireColumn.Insert()
    formula = "=(%s-%s)*%s"
    a1 = cell.Offset(2,1).GetAddress(False,False)
    a2 = cell.Offset(2,1).Address
    a3 = cell.Offset(1,2).Address
    cell.Offset(2,2).Value = formula % (a1, a2, a3)
    af_rng = cell_range(cell.Offset(2,2), cell.End(xlc.xlDown).Offset(1,2))
    cell.Offset(2,2).AutoFill(af_rng)
    af_rng.NumberFormat = "0.00"
    cell.Offset(1,2).Value = 24
    make_charts(ws, cell.Offset(1,2), all_chart)
    
def get_chart_range(cell):
    x = cell_range(cell.Offset(2,1), cell.End(xlc.xlDown))
    y = cell_range(cell.Offset(2,2), cell.End(xlc.xlDown).Offset(1,2))
    return x,y
    
def make_charts(ws, cell, all_chart):
    x,y = get_chart_range(cell)
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    CreateDataSeries(chart, x, y, cell.Offset(1,2).Value)
    CreateDataSeries(all_chart, x, y, cell.Offset(1,2).Value)
    chart.ApplyLayout(8)
    return chart

def do_import(fn, variables):
    global xl, wb, ws, cells, cell_range
    xl = Excel()
    wb = xl.Workbooks.Open(fn)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cell_range = cells.Range
    

    
    all_chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    all_chart.ApplyLayout(8)
    
    FormatChart(all_chart, None, "", "Time(hr)", "DOPV(%)")
    for v in variables:
        c = cells.Find(What=v, SearchOrder=xlc.xlByRows)
        if c is None:
            raise ValueError(v)
        process_data(ws, c, all_chart)
        
    for i, s in enumerate(all_chart.SeriesCollection()):
        if i == 0: continue
        s.AxisGroup = 2
    all_chart.Axes(xlc.xlSecondary, xlc.xlValue).HasTitle = True
    all_chart.Axes(xlc.xlSecondary, xlc.xlValue).AxisTitle.Text = "Flow Request (%)"
    all_chart.Location(xlc.xlLocationAsNewSheet)
    
    

In [60]:
variables = [
    "DOPV(%)",
    "DON2FlowActualRequest(%)",
    "DOO2FlowControllerRequestLimited(%)",
]

do_import('C:\\Users\\Nathan\\Downloads\\2017031310394614.csv', variables)

In [61]:
variables[1] = "MFCN2FlowFeedback(LPM)"
do_import('C:\\PBSCloudStation\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\PID Tuning\\DO\\Raw Data\\15L DO stabilize 100% to 50% 170309.csv', variables)

In [135]:
import numpy as np


def qf(a,b,c):
    return (-b+np.sqrt(b**2-4*a*c))/(2*a)

def ph_to_co2(ph, hco3):
    h = 10**(-ph)
    w = 1e-7
    ka = 4.45e-7
    bc = hco3 + h - w
    co2 = bc * h / ka
    return co2 / 3.4e-2

def cpH(co2, bicarb):
    # co2 as fraction of atmosphere
    c1 = co2 * 3.4e-2  # henry's law
    c2 = 1e-7
    c3 = bicarb
    ka = 4.45e-7  # pka 6.35
    a = 1
    b = c2+c3+ka
    c = c2*c3-ka*c1
    x=qf(a,b,c)
    return -np.log10(c2+x)

In [137]:
from decimal import Decimal as D
for ph in 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13:
    co2_exp = ph_to_co2(ph, 2.02/84)
    

In [119]:
m = 2.02/84
ph = 13
co2 = ph4(ph, m)

w=1e-7
h=10**(-ph)
bc = m + h - w
x = co2
h * bc / x

4.45e-07

In [123]:
x

5.403936864655966e-09

In [129]:
cpH(0, m)

11.732718352753357